In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Using OpenMDAO with MPI

In the feature notebooks for [ParallelGroup](../features/core_features/working_with_groups/parallel_group.ipynb) and 
for [Distributed Variables](../features/core_features/working_with_components/distributed_components.ipynb), you learned how to build a model that can take advantage of multiple processors to speed up your calculations. This document gives further details about how data is handled when using either of these features.

## Parallel Subsystems

The `ParallelGroup` allows you to declare that a set of unconnected `Components` or `Groups` should be run in parallel. For example, consider the model in the following diagram:

![Non-parallel example](images/Par1.png)

This model contains two components that don't depend on each other's outputs, and thus those calculations can be performed simultaneously by placing them in a `ParallelGroup`.  When a model containing a `ParallelGroup` is run without using MPI, its components are just executed in succession.  But when it is run using `mpirun` or `mpiexec`, its components are divided amongst the processors. To take fullest advantage of the available processors, the number of subsystems in the ParallelGroup should be equal to the number of available processors. However, it will still work if the number of processors is higher or lower than you need. If you don't give it enough processors, then some processors will sequentially execute some of the components. If the number of subsystems is evenly divisible by the number of processors, then you won't have idle time on any of the processors, so that is ideal. If you give your model too many processors, those processors will be idle during execution of the parallel subsystem.

The following diagram shows the same example executing on 2 processors.

![Parallel subsystem example](images/Par2.png)

We see here that every component that isn't under the ParallelGroup is executed on all processors. This is done to limit data transfer between the processors. Similarly, the input and output vectors on these components is the same on all processors. We sometimes call these duplicated variables, but it is clearer to call them non-parallel non-distributed variables.

In contrast, the inputs and outputs on the parallel components only exist on their execution processor. In this model, there are parallel outputs that need to be passed downstream to the final component.  To make this happen, OpenMDAO broadcasts them from the rank that owns them to every rank. This can be seen in the diagram as the crossed arrows that connect to x1 and x2.

Since component execution is repeated on all processors, component authors need to be careful about file operations which can collide if they are called from all processors at once. The safest way to handle these is to restrict them to only write files on the root processor. In addition, the computation of derivatives is duplicated on all processes except for the components in the parallel group, which handle their own unique parts of the calculation. 

## Distributed Components

OpenMDAO also allows you to add distributed variables to any implicit or explicit component. These can be refered to as distributed components, though there isn't a distributed component class. This feature gives you the ability to parallelize the internal calculation of your component just like the `ParallelGroup` can parallelize a larger part of the model. Distributed variables may hold different values in all processes, and can even be empty in some processes if declared as such, but non-distributed variables must have the same size and value in all processes.

